In [35]:
# using data from https://github.com/zhanglabNKU/APIN/blob/master/

import Bio
import numpy as np
import matplotlib.pyplot as plt

In [36]:
from Bio import SeqIO

codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']


def one_hot_encode(seq):
    o = list(set(codes) - set(seq))
    s = pd.DataFrame(list(seq))    
    x = pd.DataFrame(np.zeros((len(seq),len(o)),dtype=int),columns=o)    
    a = s[0].str.get_dummies(sep=',')
    a = a.join(x)
    a = a.sort_index(axis=1)
    e = a.values.flatten()
    return e

In [43]:
amps_tr=[]
amps_te=[]
amps_val=[]
decoy_tr=[]
decoy_te=[]
decoy_val=[]

y_tr=[]
y_te=[]
y_val=[]

from Bio import SeqIO

for sequence in SeqIO.parse('AMP.tr.fa', "fasta"):
    amps_tr.append(str(sequence.seq))
    y_tr.append(1)
    
for sequence in SeqIO.parse('AMP.te.fa', "fasta"):
    amps_te.append(str(sequence.seq))
    y_te.append(1)    
    
for sequence in SeqIO.parse('AMP.eval.fa', "fasta"):
    amps_val.append(str(sequence.seq))
    y_val.append(1)    
    
for sequence in SeqIO.parse('DECOY.tr.fa', "fasta"):
    decoy_tr.append(str(sequence.seq))
    y_tr.append(0)

for sequence in SeqIO.parse('DECOY.te.fa', "fasta"):
    decoy_te.append(str(sequence.seq))
    y_te.append(0)    

for sequence in SeqIO.parse('DECOY.eval.fa', "fasta"):
    decoy_val.append(str(sequence.seq))
    y_val.append(0)    

In [44]:
import numpy as np

amps_tr.extend(decoy_tr)
amps_tr.extend(amps_te)
amps_tr.extend(decoy_te)
amps_tr.extend(amps_val)
amps_tr.extend(decoy_val)

y_tr.extend(y_te)
y_tr.extend(y_val)

X=amps_tr
y=y_tr

X=np.array(X)
y=np.array(y)

In [47]:
import tensorflow
one_hot_seqs=[one_hot_encode(i) for i in X]
padded_seqs=tensorflow.keras.preprocessing.sequence.pad_sequences(one_hot_seqs,value=0.0)
X=padded_seqs

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=6)

In [57]:
X_train.shape

(2133, 1, 3660)

In [60]:
import keras

#Convolutional LSTM

NUM_CLASSES = 2
# convert class labels to one-hot encoded, should have shape (?, NUM_CLASSES)
y_train2 = keras.utils.to_categorical(y_train)
y_test2 = keras.utils.to_categorical(y_test)
# import necessary building blocks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,  Flatten, Dense, Activation, Dropout,BatchNormalization,LSTM, LeakyReLU

In [65]:
def make_model():
    """
    Define your model architecture here.
    Returns `Sequential` model.
    """
    model = Sequential()
    model.add(Conv1D(input_shape=X_train[0].shape,padding="same",kernel_size=3,filters=16))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    
    model.add(LSTM(100, activation='relu', return_sequences=True))
    
    model.add(Conv1D(padding="same",kernel_size=3,filters=32))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    

    model.add(Dropout(0.25))
    model.add(Conv1D(padding="same",kernel_size=3,filters=32))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    
    model.add(Conv1D(padding="same",kernel_size=3,filters=64))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    
    model.add(Dropout(0.25))
    model.add(Flatten())
    
    model.add(Dense(256))
    model.add(LeakyReLU(0.1))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES))
    model.add(LeakyReLU(0.1))
    
    model.add(Activation("softmax"))
    

   
    
    return model

In [66]:
model = make_model()
model.summary()

2022-08-21 23:58:23.537855: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1, 16)             175696    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 1, 16)             0         
                                                                 
 batch_normalization (BatchN  (None, 1, 16)            64        
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 1, 100)            46800     
                                                                 
 conv1d_1 (Conv1D)           (None, 1, 32)             9632      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 1, 32)             0         
                                                        

In [72]:
from keras import backend as K
INIT_LR = 5e-3  # initial learning rate
BATCH_SIZE = 32
EPOCHS = 100


# don't call K.set_learning_phase() !!! (otherwise will enable dropout in train/test simultaneously)
model = make_model()  # define our model

# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss='categorical_crossentropy',  # we train 2-way classification
    optimizer=tensorflow.keras.optimizers.Adamax(lr=INIT_LR),  # for SGD
    metrics=['accuracy']  # report accuracy during training
)

# scheduler of learning rate (decay with epochs)
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# callback for printing of actual learning rate used by optimizer
class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", K.get_value(model.optimizer.lr))

# fit model
model.fit(
    X_train, y_train2,  # prepared data
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), LrHistory()],
    validation_data=(X_test, y_test2),
    shuffle=True,
    verbose=1
)

Learning rate: 0.005
Epoch 1/100
67/67 [==============================] - 4s 17ms/step - loss: 0.6713 - accuracy: 0.6109 - val_loss: 0.6925 - val_accuracy: 0.4961 - lr: 0.0050
Learning rate: 0.0045
Epoch 2/100
67/67 [==============================] - 1s 11ms/step - loss: 0.4784 - accuracy: 0.7956 - val_loss: 0.6873 - val_accuracy: 0.7294 - lr: 0.0045
Learning rate: 0.00405
Epoch 3/100
67/67 [==============================] - 1s 10ms/step - loss: 0.3203 - accuracy: 0.8687 - val_loss: 0.6641 - val_accuracy: 0.8011 - lr: 0.0041
Learning rate: 0.003645
Epoch 4/100
67/67 [==============================] - 1s 11ms/step - loss: 0.2242 - accuracy: 0.9151 - val_loss: 0.5480 - val_accuracy: 0.7864 - lr: 0.0036
Learning rate: 0.0032805
Epoch 5/100
67/67 [==============================] - 1s 11ms/step - loss: 0.1663 - accuracy: 0.9339 - val_loss: 0.4399 - val_accuracy: 0.8250 - lr: 0.0033
Learning rate: 0.00295245
Epoch 6/100
67/67 [==============================] - 1s 10ms/step - loss: 0.1326 - a

67/67 [==============================] - 1s 11ms/step - loss: 0.0136 - accuracy: 0.9958 - val_loss: 0.9847 - val_accuracy: 0.8299 - lr: 4.8489e-05
Learning rate: 4.363982e-05
Epoch 46/100
67/67 [==============================] - 1s 11ms/step - loss: 0.0269 - accuracy: 0.9911 - val_loss: 0.9865 - val_accuracy: 0.8285 - lr: 4.3640e-05
Learning rate: 3.9275837e-05
Epoch 47/100
67/67 [==============================] - 1s 11ms/step - loss: 0.0191 - accuracy: 0.9925 - val_loss: 0.9863 - val_accuracy: 0.8292 - lr: 3.9276e-05
Learning rate: 3.5348254e-05
Epoch 48/100
67/67 [==============================] - 1s 11ms/step - loss: 0.0156 - accuracy: 0.9944 - val_loss: 0.9867 - val_accuracy: 0.8285 - lr: 3.5348e-05
Learning rate: 3.1813426e-05
Epoch 49/100
67/67 [==============================] - 1s 12ms/step - loss: 0.0161 - accuracy: 0.9948 - val_loss: 0.9850 - val_accuracy: 0.8285 - lr: 3.1813e-05
Learning rate: 2.8632085e-05
Epoch 50/100
67/67 [==============================] - 1s 11ms/step - 

67/67 [==============================] - 1s 10ms/step - loss: 0.0263 - accuracy: 0.9916 - val_loss: 0.9901 - val_accuracy: 0.8292 - lr: 5.2248e-07
Learning rate: 4.7023053e-07
Epoch 89/100
67/67 [==============================] - 1s 10ms/step - loss: 0.0122 - accuracy: 0.9967 - val_loss: 0.9943 - val_accuracy: 0.8299 - lr: 4.7023e-07
Learning rate: 4.232075e-07
Epoch 90/100
67/67 [==============================] - 1s 10ms/step - loss: 0.0106 - accuracy: 0.9967 - val_loss: 0.9957 - val_accuracy: 0.8292 - lr: 4.2321e-07
Learning rate: 3.8088675e-07
Epoch 91/100
67/67 [==============================] - 1s 10ms/step - loss: 0.0161 - accuracy: 0.9953 - val_loss: 0.9928 - val_accuracy: 0.8299 - lr: 3.8089e-07
Learning rate: 3.4279807e-07
Epoch 92/100
67/67 [==============================] - 1s 10ms/step - loss: 0.0064 - accuracy: 0.9977 - val_loss: 0.9911 - val_accuracy: 0.8292 - lr: 3.4280e-07
Learning rate: 3.0851825e-07
Epoch 93/100
67/67 [==============================] - 1s 10ms/step - 

In [74]:
test_predictions = model.predict(X_test).argmax(axis=-1)

45/45 [==============================] - 1s 6ms/step


In [75]:
test_answers = y_test2.argmax(axis=-1)

In [76]:
test_accuracy = np.mean(test_predictions==test_answers)

In [77]:
print(test_accuracy*100,'%') 

82.99367533380183 %
